In [5]:
import torch

checkpoint_path = "/root/sag_train/checkpoints/aceflow3_250401/epoch=22-step=460k_pretrained_ft_240k_small_repa.ckpt"

checkpoint = torch.load(checkpoint_path, map_location="cpu")

/tmp/ipykernel_3367140/2080906558.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location="cpu")


In [6]:
checkpoint.keys()

odict_keys(['resampler_mert.kernel', 'resampler_mhubert.kernel', 'vae.mel_transform.spectrogram.window', 'vae.mel_transform.mel_scale.fb', 'vae.vocoder.mel_transform.spectrogram.window', 'vae.vocoder.mel_transform.mel_scale.fb', 'vae.resampler.kernel', 'lyric_processor.lyric_text_model.shared.weight', 'lyric_processor.lyric_text_model.encoder.block.0.layer.0.SelfAttention.q.weight', 'lyric_processor.lyric_text_model.encoder.block.0.layer.0.SelfAttention.k.weight', 'lyric_processor.lyric_text_model.encoder.block.0.layer.0.SelfAttention.v.weight', 'lyric_processor.lyric_text_model.encoder.block.0.layer.0.SelfAttention.o.weight', 'lyric_processor.lyric_text_model.encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight', 'lyric_processor.lyric_text_model.encoder.block.0.layer.0.layer_norm.weight', 'lyric_processor.lyric_text_model.encoder.block.0.layer.1.DenseReluDense.wi_0.weight', 'lyric_processor.lyric_text_model.encoder.block.0.layer.1.DenseReluDense.wi_1.weight', 'lyric_p

In [7]:
transformers_checkpoint = {k.replace("transformers.", ""):v for k, v in checkpoint.items() if k.startswith("transformers.")}

In [1]:
from models.fusic_transformer import FusicTransformer2DModel

/root/miniconda3/envs/sag_train/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import json

with open("models/config.json") as f:
    config = json.load(f)

In [8]:
fusic_model = FusicTransformer2DModel(**config)

In [9]:
fusic_model.load_state_dict(transformers_checkpoint, strict=True)

<All keys matched successfully>

In [13]:
fusic_model = fusic_model.to(torch.bfloat16)

In [14]:
fusic_model.save_pretrained("checkpoints/fusic_transformer")

In [15]:
from diffusers import AutoencoderDC

DEFAULT_PRETRAINED_PATH = "./checkpoints/music_dcae_f8c8"
dcae = AutoencoderDC.from_pretrained(DEFAULT_PRETRAINED_PATH)

In [18]:
dcae = dcae.to(torch.bfloat16)

In [19]:
dcae.save_pretrained("checkpoints/music_dcae_f8c8")

In [1]:
from music_dcae.music_vocoder import ADaMoSHiFiGANV1

/root/miniconda3/envs/sag_train/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
vocoder = ADaMoSHiFiGANV1(checkpoint_path="/root/sag_train/checkpoints/music_vocoder.pt")

/root/miniconda3/envs/sag_train/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/vepfs-d-data/q-ace/repo/gongjunmin/ACEFlow3/music_dcae/music_vocoder.py:540: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We rec

In [5]:
import torch

vocoder = vocoder.to(torch.bfloat16)

In [6]:
vocoder.save_pretrained("checkpoints/music_vocoder")

In [1]:
from music_dcae.music_dcae_pipeline import MusicDCAE

/root/miniconda3/envs/sag_train/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
music_dcae = MusicDCAE.from_pretrained("./checkpoints/music_dcae_vocoder")

Some weights of the model checkpoint were not used when initializing MusicDCAE: 
 ['dcae.decoder.conv_in.bias, dcae.decoder.conv_in.weight, dcae.decoder.conv_out.bias, dcae.decoder.conv_out.weight, dcae.decoder.norm_out.bias, dcae.decoder.norm_out.weight, dcae.decoder.up_blocks.0.0.conv.bias, dcae.decoder.up_blocks.0.0.conv.weight, dcae.decoder.up_blocks.0.1.conv1.bias, dcae.decoder.up_blocks.0.1.conv1.weight, dcae.decoder.up_blocks.0.1.conv2.weight, dcae.decoder.up_blocks.0.1.norm.bias, dcae.decoder.up_blocks.0.1.norm.weight, dcae.decoder.up_blocks.0.2.conv1.bias, dcae.decoder.up_blocks.0.2.conv1.weight, dcae.decoder.up_blocks.0.2.conv2.weight, dcae.decoder.up_blocks.0.2.norm.bias, dcae.decoder.up_blocks.0.2.norm.weight, dcae.decoder.up_blocks.0.3.conv1.bias, dcae.decoder.up_blocks.0.3.conv1.weight, dcae.decoder.up_blocks.0.3.conv2.weight, dcae.decoder.up_blocks.0.3.norm.bias, dcae.decoder.up_blocks.0.3.norm.weight, dcae.decoder.up_blocks.1.0.conv.bias, dcae.decoder.up_blocks.1.0.conv

In [4]:
music_dcae

MusicDCAE(
  (dcae): AutoencoderDC(
    (encoder): Encoder(
      (conv_in): Conv2d(2, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (down_blocks): ModuleList(
        (0): Sequential(
          (0): ResBlock(
            (nonlinearity): SiLU()
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (norm): RMSNorm()
          )
          (1): ResBlock(
            (nonlinearity): SiLU()
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (norm): RMSNorm()
          )
          (2): DCDownBlock2d(
            (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          )
        )
        (1): Sequential(
          (0): ResBlock(
            (nonlinearity

In [2]:
music_dcae = MusicDCAE()

/root/miniconda3/envs/sag_train/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [3]:
music_dcae.dtype

torch.float32

In [4]:
import torch

music_dcae = music_dcae.to(torch.bfloat16)

In [5]:
music_dcae.save_pretrained("checkpoints/music_dcae_vocoder")